# Pareto Optimality Under Uncertainty

Now that we know what Pareto optimality is and that there's a tool to efficiently perform MOO, we have to face another problem: such undertaking is computationally expensive -- for large-scale real-world numerical model, one such forward run could take hours, let alone hundreds of thousand of runs required in MOO.

For this tutorial, we will demonstrate how we can speed up the process of MOO by using a surrogate model, without compromising the reliability of solutions. We will still be using the 2D modified Kursawe problem.

In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyemu


In [2]:
def kursawe_2d(x):
    x = np.array(x)
    obj1 = -10 * np.exp(-0.2 * np.sqrt(x[0]**2 + x[1]**2))
    obj2 = np.abs(x[0])**0.8 + 5 * np.sin(x[0]**3) + np.abs(x[1])**0.8 + 5 * np.sin(x[1]**3)
    return obj1, obj2

n_samples = 100
x1_range = np.linspace(-5, 5, int(np.sqrt(n_samples)))
x2_range = np.linspace(-5, 5, int(np.sqrt(n_samples)))
x1, x2 = np.meshgrid(x1_range, x2_range)
x1 = x1.flatten()
x2 = x2.flatten()

x1 = x1[:n_samples]
x2 = x2[:n_samples]

n_actual = min(len(x1), len(x2))
objectives = np.array([kursawe_2d([x1[i], x2[i]]) for i in range(n_actual)])
obj1 = objectives[:, 0]
obj2 = objectives[:, 1]

Let us first generate the template files we need.

In [3]:
base_d = "./demo_files/kursawe2d"
assert os.path.exists(base_d)

temp_d = "kursawe2d_demo"
if os.path.exists(temp_d):
    shutil.rmtree(temp_d)
shutil.copytree(base_d,temp_d)

'kursawe2d_demo'

We can easily find Pareto optimal set of decision variables by running PESTPP-MOU. Let's generate an initial swarm size of 30 and perform 20 iterations of MOO. To ensure that our initial population is evenly distributed and sufficiently samples the decision space, it is advisable to use Latin Hypercube Sampling (LHS).

In [4]:
# You can change the size of initial sample or the number of inner iterations if you want to try other values. 
# Just be aware that larger sample size and more inner iterations will take longer time to run.
swarm_size = 20
nmax_inner = 20

sys.path.append('./bin')
from LHS_sampler import generate_lhsstarter
bounds = np.array([[-5, 5] for i in range(2)])
generate_lhsstarter(os.path.join(temp_d,'kursawe2d_pbm_template'), seed=42, n_samples=swarm_size, n_dimensions=2, bounds=bounds) 

pst_file = os.path.join(temp_d, 'kursawe2d_pbm_template', 'kur.pst')
pst = pyemu.Pst(pst_file)
pst.pestpp_options['mou_population_size'] = swarm_size
pst.pestpp_options['mou_dv_population_file'] = 'starter.dv_pop.csv'
pst.control_data.noptmax = nmax_inner
pst.write(os.path.join(temp_d, 'kursawe2d_pbm_template', os.path.basename(pst_file)))

starter_pop = pd.read_csv(os.path.join(temp_d, "kursawe2d_pbm_template", "starter.dv_pop.csv"), index_col="real_name")
starter_pop.head()

noptmax:20, npar_adj:2, nnz_obs:2


,x1,x2
real_name,,
gen=0_member=0,-2.900058,2.001396
gen=0_member=1,-1.283856,2.516370
gen=0_member=2,2.938901,3.057931
gen=0_member=3,0.581404,-0.445164
gen=0_member=4,3.307439,-4.730684


Now that we have generated an initial population, let us run pestpp-mou...

In [5]:
num_workers = 8
tmpl_in = os.path.join(temp_d, "kursawe2d_pbm_template")
sys.path.insert(0,tmpl_in)
from forward_pbrun import ppw_worker as ppw_function
pyemu.os_utils.start_workers(tmpl_in, "../../bin/pestpp-mou", "kur.pst", num_workers = num_workers,
                             worker_root = temp_d, master_dir = os.path.join(temp_d, "pbm_run"),
                             ppw_function = ppw_function)
sys.path.remove(tmpl_in)

Let's plot the resulting Pareto front. Use the slider to observe how the swarm's position evolves in the objective space at each iteration as it searches for the Pareto front until it eventually converge. Also observe how the swarm moves in the decision space.

We will revisit this later to compare with SAMOO, which will be introduced in Part 3 of this tutorial. This will be referred to as "Complex MOO" run.

In [6]:
from ipywidgets import interact, IntSlider, HBox, VBox, Output
from IPython.display import display
from scipy.interpolate import griddata
import numpy as np
import glob
import matplotlib.pyplot as plt

run_data = pd.read_csv(os.path.join(temp_d, "pbm_run", "kur.pareto.summary.csv"))
max_gen = max(run_data['generation'])

csvfiles = sorted(glob.glob(os.path.join(temp_d, "pbm_run", "*[0-999].dv_pop.csv"), recursive=True), 
                  key=lambda x: int(x.split(".dv")[0].split(".")[1]))
all_dv_list = []
for file in csvfiles:
    generation = int(file.split(".dv")[0].split(".")[1])
    df = pd.read_csv(file).assign(generation=generation)
    df = df[['generation'] + [col for col in df.columns if col != 'generation']] 
    all_dv_list.append(df)
all_dv = pd.concat(all_dv_list, ignore_index=True)

csvfiles = sorted(glob.glob(os.path.join(temp_d, "pbm_run", "*[0-999].obs_pop.csv"), recursive=True), 
                      key=lambda x: int(x.split(".obs")[0].split(".")[1]))

all_obs_list = []
for file in csvfiles:
    generation = int(file.split(".obs")[0].split(".")[1])
    df = pd.read_csv(file).assign(generation=generation)
    df = df[['generation'] + [col for col in df.columns if col != 'generation']] 
    all_obs_list.append(df)
all_obs = pd.concat(all_obs_list, ignore_index=True)

all_data = pd.merge(all_dv, all_obs, on=['generation', 'real_name'])
max_gen = max(all_data['generation'])

n_samples = 100
x1_range = np.linspace(-5, 5, int(np.sqrt(n_samples)))
x2_range = np.linspace(-5, 5, int(np.sqrt(n_samples)))
x1, x2 = np.meshgrid(x1_range, x2_range)
x1 = x1.flatten()
x2 = x2.flatten()

x1 = x1[:n_samples]
x2 = x2[:n_samples]

n_actual = min(len(x1), len(x2))
objectives = np.array([kursawe_2d([x1[i], x2[i]]) for i in range(n_actual)])
obj1 = objectives[:, 0]
obj2 = objectives[:, 1]

xi = np.linspace(min(x1), max(x1), n_samples)
yi = np.linspace(min(x2), max(x2), n_samples)
xi_grid, yi_grid = np.meshgrid(xi, yi)
zi1 = griddata((x1, x2), obj1, (xi_grid, yi_grid), method='cubic')
zi2 = griddata((x1, x2), obj2, (xi_grid, yi_grid), method='cubic')

out_pareto = Output()
out_obj_space = Output()
out_contour = Output()

true_pareto = pd.read_csv(os.path.join('kursawe2d_demo', 'kursawe2d_solution_obj.csv'))

def moo_plot_all(generation):
    with out_pareto:
        out_pareto.clear_output(wait=True)
        pareto = run_data.loc[(run_data['generation']==generation) & (run_data['nsga2_front'] == 1)]
        plt.figure(figsize=(5, 6))
        plt.scatter(pareto['obj1'], pareto['obj2'], c='firebrick', s=30, alpha=0.7, label = 'current non-dominated positions')
        plt.scatter(true_pareto['obj1'], true_pareto['obj2'], c='deepskyblue', s=20, zorder = -10, label='Pareto optimal solutions')
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.xlim((-10.5, -2))
        plt.ylim((-10, 15))
        plt.legend()
        plt.title(f'Pareto Front at Generation {generation}')
        plt.tight_layout()
        plt.show()

    with out_obj_space:
        out_obj_space.clear_output(wait=True)
        all_points = run_data.loc[(run_data['generation']==generation)]
        plt.figure(figsize=(5, 6))
        plt.scatter(all_points['obj1'], all_points['obj2'], edgecolor='green', c = 'none', s=30, alpha=0.7, label = 'all candidate positions')
        plt.scatter(true_pareto['obj1'], true_pareto['obj2'], c='deepskyblue', s=20, zorder = -10, label='Pareto optimal solutions')
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.xlim((-10.5, -2))
        plt.ylim((-10, 15))
        plt.legend()
        plt.title(f'Objective Space at Generation {generation}')
        plt.tight_layout()
        plt.show()

    # with out_contour:
    #     out_contour.clear_output(wait=True)
    #     fig, ax = plt.subplots(figsize=(8, 6))
    #     contour1 = ax.contourf(xi_grid, yi_grid, zi2, 15, cmap='plasma', alpha=0.5)
    #     fig.colorbar(contour1, ax=ax, label='Objective 1')
    #     contour2 = ax.contour(xi_grid, yi_grid, zi1, 15, cmap='viridis', alpha=0.7)
    #     fig.colorbar(contour2, ax=ax, label='Objective 2')

    #     pareto = run_data.loc[(run_data['generation']==generation) & (run_data['nsga2_front'] == 1)]
    #     pareto_members = pareto['member'].values
    #     pareto_dv = all_data[all_data['real_name'].isin(pareto_members)]
    #     ax.scatter(pareto_dv['x1'], pareto_dv['x2'], edgecolor='firebrick', facecolor='none', s=40, alpha=0.8)
        
    #     ax.set_xlabel('x1')
    #     ax.set_ylabel('x2')
    #     ax.set_title(f'Pareto Optimal Decisions at Generation {generation}')
    #     ax.grid(True)
    #     plt.tight_layout()
    #     plt.show()

generation_slider = IntSlider(min=0, max=max_gen, step=1, value=max_gen, 
                             description='Generation:',
                             continuous_update=False)
moo_plot_all(max_gen)

generation_slider.observe(lambda change: moo_plot_all(change['new']), names='value')
display(VBox([
    generation_slider,
    HBox([out_obj_space, out_pareto]),
    out_contour
]))


As this benchmark is not too complex, it doesn't take much time to complete one iteration and it doesn't take a lot of iteration to obtain a set of Pareto optimal solutions. However, in reality, real-world models do not run this fast. Even with parallelisation, it can take a few hundred, even thousand iterations until the swarm converges to the Pareto front. If important timely decisions rely on the outcome of such optimisation process, we would need some help to speed up this process. 

Let's pretend for a while that the Fonseca-Fleming problem is a complex model that is expensive to evaluate. There's nothing we could do about the run time of the complex model. What we can do though is to employ an "emulator", a faster and cheaper model that approximates the relationships between the known (i.e., previously evaluated) input and outputs of the complex model in order to predict its response to some input values that were not previously evaluated. This emulator is also known as the surrogate model. Because the surrogate model is an approximation, its prediction has errors. These errors could mislead the decision-making to some suboptimal solutions that are not actually in the Pareto front. This is some price to pay for speeding up the process, but don't worry as we have means to manage these uncertainties and still obtain truly Pareto optimal solutions.

There are many surrogate models available in literature. However, we will use Gaussian Process Regression (GPR) as it provides a convenient way to take into account the uncertainty in predictions of the surrogate model, which, as previously said, needs to be managed well.

Let's take a quick detour to see how GPR works. Here's a nonlinear function with 1D input and output. We generate 10 training points composed of the inputs and corresponding outputs when evaluated in the nonlinear function. We then use the training points to predict the output of the nonlinear function at some test points.

In [10]:
import laGPy as gpr
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Generate some training data with a non-linear function
np.random.seed(42)
X_train = np.sort(np.random.uniform(-3, 3, 10))
# True function: f(x) = sin(x) + 0.1*x^2
y_train = np.sin(X_train) + 0.1 * np.square(X_train) + np.random.normal(0, 0.1, X_train.shape)

gp = gpr.buildGP(X_train, y_train, wdir = './kursawe2d_demo')

# Create test points for prediction
X_test = np.linspace(-4, 4, 20)[:, np.newaxis]

# Process each row in X_test individually and store results in arrays
loaded_gp = gp.loadGP()
mean_pred = np.zeros(X_test.shape[0])
std_pred = np.zeros(X_test.shape[0])

for i in range(X_test.shape[0]):
    X_row = X_test[i:i+1]  # Get single row as 2D array
    sims = loaded_gp.predict_lite(X_row)
    mean_pred[i] = sims['mean']
    std_pred[i] = np.sqrt(sims['s2'])

# Plot the results
plt.figure(figsize=(10, 6))

# Plot training points
plt.scatter(X_train, y_train, color='red', label='Training points', s=50, zorder=3)

# Plot the mean prediction
plt.plot(X_test, mean_pred, color='blue', label='Mean prediction')

# Plot the confidence interval (±2 standard deviations, 95% confidence)
plt.fill_between(X_test.ravel(), 
                 mean_pred - 2 * std_pred, 
                 mean_pred + 2 * std_pred, 
                 color='blue', alpha=0.2, label='95% confidence interval')

# Plot the true function for comparison
true_y = np.sin(X_test) + 0.1 * np.square(X_test)
plt.plot(X_test, true_y, color='green', linestyle='--', label='True function')

plt.title('Gaussian Process Regression Example')
plt.xlabel('Input (x)')
plt.ylabel('Output (y)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Display the learned kernel parameters
print("Learned kernel parameters:")
print(f"Length scale: {gp.kernel_.length_scale}")



IndexError: cannot do a non-empty take from an empty axes.